In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"
BOOKMAKER_URL = "https://api.sportmonks.com/v3/odds/bookmakers"  # bookmaker lookup is under /v3/odds/...

# -------------------------
# Fixtures (next 12 hours)
# -------------------------
def fetch_fixtures_next_12h() -> pd.DataFrame:
    """Return fixtures starting in the next 12 hours (UTC)."""
    now_utc = datetime.now(timezone.utc)
    end_utc = now_utc + timedelta(hours=12)

    start_date = now_utc.date()
    end_date = (now_utc + timedelta(days=1)).date()
    endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"

    session = requests.Session()
    fixtures = []
    page = 1

    while True:
        r = session.get(
            f"{BASE_URL}{endpoint}",
            params={"api_token": API_TOKEN, "page": page},
            timeout=30,
        )
        r.raise_for_status()
        payload = r.json()

        fixtures.extend(payload.get("data") or [])

        # Pagination handling: be defensive across endpoints/plans
        pagination = ((payload.get("meta") or {}).get("pagination")) or {}
        has_more = pagination.get("has_more")
        total_pages = pagination.get("total_pages")
        current_page = pagination.get("current_page")

        if has_more is False:
            break
        if total_pages is not None and current_page is not None and int(current_page) >= int(total_pages):
            break
        if has_more is None and (total_pages is None or current_page is None):
            # If the API didn't return pagination metadata, assume single page.
            break

        page += 1

    df = pd.DataFrame(
        {
            "FixtureID": [f.get("id") for f in fixtures],
            "Fixture": [f.get("name") for f in fixtures],
            "StartingAt": [f.get("starting_at") for f in fixtures],
        }
    )

    if df.empty:
        return df

    df["StartingAt"] = pd.to_datetime(df["StartingAt"], utc=True, errors="coerce")
    df = df[df["StartingAt"].between(now_utc, end_utc)].reset_index(drop=True)
    return df


# -------------------------
# Bookmaker ID -> Name lookup (cached)
# -------------------------
def _get_bookmaker_name(bookmaker_id: int | None, session: requests.Session, cache: dict[int, str]) -> str | None:
    """
    Resolve bookmaker_id to bookmaker name via /v3/odds/bookmakers/{ID}.
    Caches results to avoid repeated calls.
    """
    if bookmaker_id is None:
        return None

    try:
        bookmaker_id_int = int(bookmaker_id)
    except (TypeError, ValueError):
        return None

    if bookmaker_id_int in cache:
        return cache[bookmaker_id_int]

    r = session.get(
        f"{BOOKMAKER_URL}/{bookmaker_id_int}",
        params={"api_token": API_TOKEN},
        timeout=30,
    )
    if r.status_code == 404:
        return None
    r.raise_for_status()

    data = (r.json() or {}).get("data") or {}
    name = data.get("name")
    if isinstance(name, str) and name.strip():
        cache[bookmaker_id_int] = name.strip()
        return cache[bookmaker_id_int]

    return None


# -------------------------
# Odds (keep ONLY the fields used in your first version)
# -------------------------
def fetch_odds_prematch_for_fixture(
    fixture_id: int,
    session: requests.Session,
    bookmaker_cache: dict[int, str],
) -> list[dict]:
    """Pull odds for a single fixture and shape the columns we need (first-code schema)."""
    endpoint = f"/odds/pre-match/fixtures/{fixture_id}"

    r = session.get(
        f"{BASE_URL}{endpoint}",
        params={"api_token": API_TOKEN},
        timeout=30,
    )
    if r.status_code == 404:
        return []

    r.raise_for_status()
    odds = r.json().get("data") or []

    rows = []
    for odd in odds:
        bookmaker_id = odd.get("bookmaker_id")
        bookmaker_name = _get_bookmaker_name(bookmaker_id, session, bookmaker_cache)

        rows.append(
            {
                "OddID": odd.get("id"),
                "FixtureID": odd.get("fixture_id"),
                "MarketID": odd.get("market_id"),
                "Bookmaker": bookmaker_name,  # <-- filled reliably via bookmaker_id lookup
                "Market": odd.get("market_description"),
                "Label": odd.get("label"),
                "Name": odd.get("name"),
                "Value": odd.get("value"),
                "DP3": odd.get("dp3"),
                "Total": odd.get("total"),
                "Handicap": odd.get("handicap"),
                "LatestBookmakerUpdate": odd.get("latest_bookmaker_update"),
            }
        )

    return rows


def fetch_odds_next_12h() -> pd.DataFrame:
    fixtures_df = fetch_fixtures_next_12h()
    if fixtures_df.empty:
        return pd.DataFrame()

    session = requests.Session()
    bookmaker_cache: dict[int, str] = {}
    rows: list[dict] = []

    for fid in fixtures_df["FixtureID"].astype(int):
        rows.extend(fetch_odds_prematch_for_fixture(fid, session, bookmaker_cache))

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df = df.merge(fixtures_df, on="FixtureID", how="left")

    # Normalize types
    df["Odds"] = pd.to_numeric(df["Value"], errors="coerce")
    df["Total"] = pd.to_numeric(df["Total"], errors="coerce")
    df["Handicap"] = pd.to_numeric(df["Handicap"], errors="coerce")

    # Drop unusable odds (same spirit as your original)
    df = df.dropna(subset=["Odds", "MarketID", "Bookmaker", "Label", "StartingAt"]).reset_index(drop=True)

    return df


# -------------------------
# Example usage
# -------------------------
df = fetch_odds_next_12h()

columns_to_show = [
    "StartingAt",
    "Fixture",
    "Bookmaker",
    "Market",
    "Label",
    "Odds",
    "Name",
    "Total",
    "Handicap",
    "LatestBookmakerUpdate",
]

df[columns_to_show].head(50)

,StartingAt,Fixture,Bookmaker,Market,Label,Odds,Name,Total,Handicap,LatestBookmakerUpdate
0,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Team Shots,1,1.83,None,NaN,NaN,2026-01-04 19:40:37
1,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Team Shots,1,1.83,None,NaN,NaN,2026-01-04 19:40:37
2,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,1st Half Goal Line,Under,1.72,1.0,1.0,NaN,2026-01-06 16:10:37
3,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Alternative 1st Half Goal Line,Under,1.67,1.0,1.0,NaN,2026-01-06 13:40:44
4,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Player Shots,4.5,11.00,Santiago Pierotti,NaN,NaN,2026-01-06 16:10:37
5,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Player Shots,4.5,21.00,Tete Morente,NaN,NaN,2026-01-06 16:10:37
6,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Goalscorers,Anytime,8.50,Muhammed Bah,NaN,NaN,2026-01-06 16:10:37
7,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Goalscorers,First,34.00,Corrie Ndaba,NaN,NaN,2026-01-06 16:10:37
8,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Goalscorers,First,41.00,Matias Perez,NaN,NaN,2026-01-06 16:10:37
9,2026-01-06 17:00:00+00:00,Lecce vs Roma,bet365,Goalscorers,Last,29.00,Jose Angelino,NaN,NaN,2026-01-06 16:10:37


In [4]:
import pandas as pd
import numpy as np

def find_valuebets(
    df: pd.DataFrame,
    pct_threshold: float = 0.30,
    min_bookmakers: int = 3,
    group_cols: list[str] | None = None,
    odds_col: str = "Odds",
    bookmaker_col: str = "Bookmaker",
    # normalization so 1.5 vs 1.50 don't split groups:
    line_round: int = 3,
    na_sentinel: str = "__NA__",
    # optional: avoid multiple rows per bookmaker for same selection
    dedupe_per_bookmaker: bool = True,
    dedupe_keep: str = "best",  # "best" (max odds) or "latest"
    latest_col: str = "LatestBookmakerUpdate",
) -> pd.DataFrame:
    """
    Valuebet if Odds >= (1 + pct_threshold) * median(Odds) within the same selection group.

    Grouping default (as you requested):
      FixtureID, MarketID, Label, Name, Total, Handicap

    Adds:
      MedianOdds, ValueThreshold, IsValueBet, ComparedCount, ComparedBookmakers, ComparedOdds
    """
    out = df.copy()

    if group_cols is None:
        group_cols = ["FixtureID", "MarketID", "Label", "Name", "Total", "Handicap"]

    # --- Required columns check ---
    required = [odds_col, bookmaker_col] + group_cols
    missing = [c for c in required if c not in out.columns]
    if missing:
        raise KeyError(f"Missing required columns: {missing}")

    # --- Normalize core types ---
    out[odds_col] = pd.to_numeric(out[odds_col], errors="coerce")
    out[bookmaker_col] = out[bookmaker_col].astype("string")

    # Total/Handicap should be numeric for stable rounding/grouping
    # (we'll later convert to a string key with sentinel)
    out["Total"] = pd.to_numeric(out["Total"], errors="coerce").round(line_round)
    out["Handicap"] = pd.to_numeric(out["Handicap"], errors="coerce").round(line_round)

    # Name can be missing; keep as string
    out["Name"] = out["Name"].astype("string")

    # --- Drop only truly unusable rows ---
    # We DO NOT drop on Total/Handicap/Name; those can be NaN.
    out = out.dropna(subset=[odds_col, bookmaker_col, "FixtureID", "MarketID", "Label"]).copy()

    # --- Build safe grouping keys (so NaNs don't delete rows) ---
    # Create grouping versions of Name/Total/Handicap that are never null.
    out["_NameKey"] = out["Name"].fillna(na_sentinel)
    out["_TotalKey"] = out["Total"].map(lambda x: f"{x:.{line_round}f}" if pd.notna(x) else na_sentinel)
    out["_HandicapKey"] = out["Handicap"].map(lambda x: f"{x:.{line_round}f}" if pd.notna(x) else na_sentinel)

    # Use exactly the grouping you want, but with safe keys for nullable cols
    group_key_cols = ["FixtureID", "MarketID", "Label", "_NameKey", "_TotalKey", "_HandicapKey"]

    # --- Optional: dedupe (group + bookmaker) to avoid duplicates blowing up comparisons ---
    if dedupe_per_bookmaker:
        dedupe_keys = group_key_cols + [bookmaker_col]

        if dedupe_keep.lower() == "latest" and latest_col in out.columns:
            out[latest_col] = pd.to_datetime(out[latest_col], utc=True, errors="coerce")
            out = out.sort_values(by=dedupe_keys + [latest_col], ascending=True, kind="mergesort")
            out = out.drop_duplicates(subset=dedupe_keys, keep="last").copy()
        else:
            # keep "best" odds per bookmaker per group
            out = out.sort_values(by=dedupe_keys + [odds_col], ascending=True, kind="mergesort")
            out = out.drop_duplicates(subset=dedupe_keys, keep="last").copy()

    # --- Core valuebet stats ---
    g = out.groupby(group_key_cols, dropna=False)
    out["MedianOdds"] = g[odds_col].transform("median")
    out["ValueThreshold"] = out["MedianOdds"] * (1.0 + pct_threshold)
    out["_GroupBookmakersN"] = g[bookmaker_col].transform("nunique")
    out["IsValueBet"] = (out[odds_col] >= out["ValueThreshold"]) & (out["_GroupBookmakersN"] >= min_bookmakers)

    # --- Compared vs lists WITHOUT apply(axis=1) ---
    out = out.reset_index(drop=False).rename(columns={"index": "_RowID"})

    left = out[["_RowID"] + group_key_cols + [bookmaker_col, odds_col]]
    right = out[["_RowID"] + group_key_cols + [bookmaker_col, odds_col]].rename(
        columns={"_RowID": "_RowID_other", bookmaker_col: "Bookmaker_other", odds_col: "Odds_other"}
    )

    m = left.merge(right, on=group_key_cols, how="left")
    m = m[(m["_RowID"] != m["_RowID_other"]) & (m[bookmaker_col] != m["Bookmaker_other"])]

    agg = m.groupby("_RowID", sort=False).agg(
        ComparedBookmakers=("Bookmaker_other", lambda s: sorted(pd.unique(s.dropna()))),
        ComparedOdds=("Odds_other", lambda s: list(pd.to_numeric(s, errors="coerce").dropna().astype(float))),
        ComparedCount=("Bookmaker_other", lambda s: int(pd.Series(s.dropna()).nunique())),
    )

    out = out.merge(agg, on="_RowID", how="left")
    out["ComparedBookmakers"] = out["ComparedBookmakers"].apply(lambda x: x if isinstance(x, list) else [])
    out["ComparedOdds"] = out["ComparedOdds"].apply(lambda x: x if isinstance(x, list) else [])
    out["ComparedCount"] = out["ComparedCount"].fillna(0).astype(int)

    # Cleanup temp columns
    out = out.drop(columns=["_GroupBookmakersN", "_NameKey", "_TotalKey", "_HandicapKey"])

    return out


In [6]:
value_df = find_valuebets(
    df,
    pct_threshold=0.50,
    min_bookmakers=3,
    group_cols=["FixtureID", "MarketID", "Label", "Name", "Total", "Handicap"],
)

cols = [
    "StartingAt","Fixture","Bookmaker","Market","Label","Name","Total","Handicap",
    "Odds","MedianOdds","ValueThreshold","IsValueBet",
    "ComparedCount","ComparedBookmakers",
]
value_df[value_df["IsValueBet"]][cols].head(50)


,StartingAt,Fixture,Bookmaker,Market,Label,Name,Total,Handicap,Odds,MedianOdds,ValueThreshold,IsValueBet,ComparedCount,ComparedBookmakers
1186,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,3:2,3:2,NaN,NaN,81.64,51.000,76.5000,True,6,"[10Bet, 1xbet, Marathonbet, Pinnacle, Unibet, ..."
1193,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,3:3,3:3,NaN,NaN,148.84,71.000,106.5000,True,6,"[10Bet, 1xbet, Marathonbet, Pinnacle, Unibet, ..."
1199,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,3:4,3:4,NaN,NaN,240.04,101.000,151.5000,True,4,"[10Bet, 1xbet, Pinnacle, WilliamHill]"
1211,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,4:0,4:0,NaN,NaN,240.04,113.000,169.5000,True,3,"[1xbet, Pinnacle, WilliamHill]"
1216,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,4:1,4:1,NaN,NaN,240.04,101.000,151.5000,True,4,"[10Bet, 1xbet, Pinnacle, WilliamHill]"
1222,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,4:2,4:2,NaN,NaN,240.04,126.000,189.0000,True,4,"[10Bet, 1xbet, Pinnacle, WilliamHill]"
1226,2026-01-06 17:00:00+00:00,Lecce vs Roma,Sbo,Correct Score,4:3,4:3,NaN,NaN,240.04,138.500,207.7500,True,3,"[1xbet, Pinnacle, WilliamHill]"
1260,2026-01-06 17:00:00+00:00,Lecce vs Roma,Pinnacle,Correct Score,6:1,6:1,NaN,NaN,251.00,151.000,226.5000,True,2,"[1xbet, WilliamHill]"
3908,2026-01-06 19:45:00+00:00,Sassuolo vs Juventus,MelBet,Asian Handicap,Away,Away,NaN,-1.50,2.86,1.030,1.5450,True,2,"[1xbet, Marathonbet]"
4996,2026-01-06 19:45:00+00:00,Sassuolo vs Juventus,Sbo,Correct Score,3:3,3:3,NaN,NaN,105.64,67.000,100.5000,True,6,"[10Bet, 1xbet, Marathonbet, Pinnacle, Unibet, ..."


In [7]:
# only show valuebets:
value_bets = value_df[value_df["IsValueBet"]]

# remove columns: DP3, OddID, _RowID, MarketID
value_bets = value_bets.drop(columns=["DP3", "OddID", "_RowID", "MarketID"], errors="ignore")
value_bets.head(50)


,FixtureID,Bookmaker,Market,Label,Name,Value,Total,Handicap,LatestBookmakerUpdate,Fixture,StartingAt,Odds,MedianOdds,ValueThreshold,IsValueBet,ComparedBookmakers,ComparedOdds,ComparedCount
1186,19425064,Sbo,Correct Score,3:2,3:2,81.64,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,81.64,51.000,76.5000,True,"[10Bet, 1xbet, Marathonbet, Pinnacle, Unibet, ...","[46.0, 60.0, 64.0, 51.0, 51.0, 51.0]",6
1193,19425064,Sbo,Correct Score,3:3,3:3,148.84,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,148.84,71.000,106.5000,True,"[10Bet, 1xbet, Marathonbet, Pinnacle, Unibet, ...","[67.0, 90.0, 96.0, 67.0, 67.0, 71.0]",6
1199,19425064,Sbo,Correct Score,3:4,3:4,240.04,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,240.04,101.000,151.5000,True,"[10Bet, 1xbet, Pinnacle, WilliamHill]","[126.0, 100.0, 91.0, 101.0]",4
1211,19425064,Sbo,Correct Score,4:0,4:0,240.04,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,240.04,113.000,169.5000,True,"[1xbet, Pinnacle, WilliamHill]","[100.0, 81.0, 126.0]",3
1216,19425064,Sbo,Correct Score,4:1,4:1,240.04,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,240.04,101.000,151.5000,True,"[10Bet, 1xbet, Pinnacle, WilliamHill]","[126.0, 100.0, 71.0, 101.0]",4
1222,19425064,Sbo,Correct Score,4:2,4:2,240.04,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,240.04,126.000,189.0000,True,"[10Bet, 1xbet, Pinnacle, WilliamHill]","[126.0, 100.0, 101.0, 126.0]",4
1226,19425064,Sbo,Correct Score,4:3,4:3,240.04,NaN,NaN,2026-01-06 06:01:45,Lecce vs Roma,2026-01-06 17:00:00+00:00,240.04,138.500,207.7500,True,"[1xbet, Pinnacle, WilliamHill]","[100.0, 126.0, 151.0]",3
1260,19425064,Pinnacle,Correct Score,6:1,6:1,251.00,NaN,NaN,2026-01-06 07:30:03,Lecce vs Roma,2026-01-06 17:00:00+00:00,251.00,151.000,226.5000,True,"[1xbet, WilliamHill]","[100.0, 151.0]",2
3908,19425069,MelBet,Asian Handicap,Away,Away,2.86,NaN,-1.50,2026-01-06 16:00:35,Sassuolo vs Juventus,2026-01-06 19:45:00+00:00,2.86,1.030,1.5450,True,"[1xbet, Marathonbet]","[1.03, 1.0]",2
4996,19425069,Sbo,Correct Score,3:3,3:3,105.64,NaN,NaN,2026-01-06 05:12:25,Sassuolo vs Juventus,2026-01-06 19:45:00+00:00,105.64,67.000,100.5000,True,"[10Bet, 1xbet, Marathonbet, Pinnacle, Unibet, ...","[56.0, 65.0, 71.0, 74.09, 51.0, 67.0]",6


In [9]:
# 1) Do you have enough rows after cleaning?
print("Rows:", len(value_df))
print("Valuebets:", value_df["IsValueBet"].sum())

# 2) How many groups have >= 3 bookmakers?
tmp = df.copy()
tmp["Odds"] = pd.to_numeric(tmp["Odds"], errors="coerce")
tmp = tmp.dropna(subset=["Odds", "Bookmaker", "FixtureID", "MarketID", "Label", "Name", "Total", "Handicap"])

grp = tmp.groupby(["FixtureID","MarketID","Label","Name","Total","Handicap"])["Bookmaker"].nunique()
print("Groups total:", len(grp))
print("Groups with >=3 books:", (grp >= 3).sum())

# 3) What's the maximum "value ratio" you even see?
# ratio = Odds / MedianOdds; valuebet at 30% is ratio >= 1.30
mx = (value_df["Odds"] / value_df["MedianOdds"]).max()
print("Max Odds/MedianOdds ratio:", mx)

Rows: 0
Valuebets: 0
Groups total: 0
Groups with >=3 books: 0
Max Odds/MedianOdds ratio: nan
